In [ ]:
%pip install torch transformers datasets scikit-learn flask

   ---------------------------------------- 0.0/484.9 kB ? eta -:--:--
   ----- ---------------------------------- 61.4/484.9 kB 1.7 MB/s eta 0:00:01
   ---------- ----------------------------- 122.9/484.9 kB 1.4 MB/s eta 0:00:01
   ------------------ --------------------- 225.3/484.9 kB 2.3 MB/s eta 0:00:01
   ---------------------------------------- 484.9/484.9 kB 2.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/146.7 kB ? eta -:--:--
   ---------------------------------------- 146.7/146.7 kB 8.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/25.3 MB ? eta -:--:--
   ---------------------------------------- 0.2/25.3 MB 14.7 MB/s eta 0:00:02
    --------------------------------------- 0.4/25.3 MB 4.4 MB/s eta 0:00:06
    --------------------------------------- 0.5/25.3 MB 4.4 MB/s eta 0:00:06
   - -------------------------------------- 1.0/25.3 MB 5.8 MB/s eta 0:00:05
   - -------------------------------------- 1.2/25.3 MB 6.0 MB/s eta 0:00:05
   

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.32.0 requires packaging<24,>=16.8, but you have packaging 24.1 which is incompatible.


In [2]:
import pandas as pd

# Load dataset
df = pd.read_csv("train.csv")  # Replace with actual path

# Display the first few rows
print(df.head())

                 id                                       comment_text  toxic  \
0  0000997932d777bf  Explanation\nWhy the edits made under my usern...      0   
1  000103f0d9cfb60f  D'aww! He matches this background colour I'm s...      0   
2  000113f07ec002fd  Hey man, I'm really not trying to edit war. It...      0   
3  0001b41b1c6bb37e  "\nMore\nI can't make any real suggestions on ...      0   
4  0001d958c54c6e35  You, sir, are my hero. Any chance you remember...      0   

   severe_toxic  obscene  threat  insult  identity_hate  
0             0        0       0       0              0  
1             0        0       0       0              0  
2             0        0       0       0              0  
3             0        0       0       0              0  
4             0        0       0       0              0  


In [3]:
import re

def clean_text(text):
    text = str(text).lower()  # Convert to lowercase
    text = re.sub(r"\n", " ", text)  # Remove newline characters
    text = re.sub(r"https?://\S+|www\.\S+", "", text)  # Remove URLs
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)  # Remove special characters
    return text

df["cleaned_text"] = df["comment_text"].apply(clean_text)

# Show cleaned text
print(df[["comment_text", "cleaned_text"]].head())

                                        comment_text  \
0  Explanation\nWhy the edits made under my usern...   
1  D'aww! He matches this background colour I'm s...   
2  Hey man, I'm really not trying to edit war. It...   
3  "\nMore\nI can't make any real suggestions on ...   
4  You, sir, are my hero. Any chance you remember...   

                                        cleaned_text  
0  explanation why the edits made under my userna...  
1  daww he matches this background colour im seem...  
2  hey man im really not trying to edit war its j...  
3   more i cant make any real suggestions on impr...  
4  you sir are my hero any chance you remember wh...  


In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_texts(texts):
    return tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

# Example tokenization
tokens = tokenize_texts(df["cleaned_text"][:5].tolist())
print(tokens)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

c:\Users\Lenovo\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Lenovo\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

{'input_ids': tensor([[  101,  7526,  2339,  1996, 10086,  2015,  2081,  2104,  2026,  5310,
         18442, 13076, 12392,  2050,  5470,  2020, 16407,  2027,  4694,  2102,
          3158,  9305, 22556,  2074,  8503,  2006,  2070,  3806,  2044,  1045,
          5444,  2012,  2047,  2259, 14421,  6904,  2278,  1998,  3531,  2123,
          2102,  6366,  1996, 23561,  2013,  1996,  2831,  3931,  2144, 10047,
          3394,  2085,  2620,  2683, 11387, 22275,  2620, 22907,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

In [5]:
import torch
from sklearn.model_selection import train_test_split

# Select labels
labels = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
X_train, X_test, y_train, y_test = train_test_split(df["cleaned_text"], df[labels], test_size=0.2, random_state=42)

# Convert labels to tensor format
y_train = torch.tensor(y_train.values, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.float32)

In [6]:
from transformers import AutoModelForSequenceClassification

# Load BERT model with multi-label output
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(labels))

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load DistilBERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("unitary/toxic-bert")

def predict_toxicity(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    scores = torch.sigmoid(outputs.logits).detach().numpy()

    labels = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
    predictions = {labels[i]: float(scores[0][i]) for i in range(len(labels))}

    return predictions

# Example
comment = "You are an idiot!"
print(predict_toxicity(comment))

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

c:\Users\Lenovo\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Lenovo\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/811 [00:00<?, ?B/s]

c:\Users\Lenovo\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Lenovo\.cache\huggingface\hub\models--unitary--toxic-bert. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

{'toxic': 0.9886645078659058, 'severe_toxic': 0.04414261505007744, 'obscene': 0.7792555689811707, 'threat': 0.0012983783381059766, 'insult': 0.9576008319854736, 'identity_hate': 0.013248424045741558}


In [8]:
print(predict_toxicity("Stone Sour sucks anus"))

{'toxic': 0.9903157353401184, 'severe_toxic': 0.08822569251060486, 'obscene': 0.9641202688217163, 'threat': 0.0016730638453736901, 'insult': 0.6229175925254822, 'identity_hate': 0.0064499881118535995}


In [9]:
print(predict_toxicity("I WILL BURN YOU TO HELL IF YOU REVOKE MY TALK PAGE ACCESS!!!!!!!!!!!!!"))

{'toxic': 0.9419159293174744, 'severe_toxic': 0.14494219422340393, 'obscene': 0.1890186369419098, 'threat': 0.8322195410728455, 'insult': 0.1898946613073349, 'identity_hate': 0.025236887857317924}


In [10]:
print(predict_toxicity("aapn bhtla aanand jhala.."))

{'toxic': 0.009629295207560062, 'severe_toxic': 0.00010161249520024285, 'obscene': 0.0005881971446797252, 'threat': 0.00013783076428808272, 'insult': 0.00048072938807308674, 'identity_hate': 0.00022104597883298993}


In [11]:
print(predict_toxicity("Beautiful"))

{'toxic': 0.001159388804808259, 'severe_toxic': 0.00010512669541640207, 'obscene': 0.0001907793339341879, 'threat': 0.0001013056025840342, 'insult': 0.000186040997505188, 'identity_hate': 0.0001519394136266783}
